<a href="https://colab.research.google.com/github/jelittle/4413-final-project/blob/colour-simulation/CarColourSimulation/ultralytics_yolo_car_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dataset Structure

The data distribution within the Carparts Segmentation Dataset is organized as outlined below:

- **Training set**: Includes 3156 images, each accompanied by its corresponding annotations.
- **Testing set**: Comprises 276 images, with each one paired with its respective annotations.
- **Validation set**: Consists of 401 images, each having corresponding annotations.

## Setup

pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/pyproject.toml) and check software and hardware.

[![PyPI - Version](https://img.shields.io/pypi/v/ultralytics?logo=pypi&logoColor=white)](https://pypi.org/project/ultralytics/) [![Downloads](https://static.pepy.tech/badge/ultralytics)](https://www.pepy.tech/projects/ultralytics) [![PyPI - Python Version](https://img.shields.io/pypi/pyversions/ultralytics?logo=python&logoColor=gold)](https://pypi.org/project/ultralytics/)

In [5]:
!pip install ultralytics
!pip install --upgrade dataset-tools
!pip install supervisely

In [2]:

import ultralytics
ultralytics.checks()

Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 42.6/112.6 GB disk)


download dataset and convert dataset to yolo format



In [4]:
import dataset_tools as dtools

dtools.download(dataset='Car Segmentation', dst_dir='~/dataset-ninja/')






Unpacking 'car-segmentation.tar': 100%|██████████| 429/429 [00:01<00:00, 360.58file/s] 


'/root/dataset-ninja/car-segmentation'

In [21]:
import supervisely as sly
import os

dataset_path = '/root/dataset-ninja/car-segmentation'
yolo_folder = '/root/yolo_dataset1'
yolo_yaml = '/root/yolo_dataset1/data_config.yaml'
# Ensure the output directory exists
os.makedirs(yolo_folder, exist_ok=True)

# Convert the dataset to YOLO format
sly.convert.to_yolo(dataset_path, yolo_folder,  task_type="segment")

print(f"Dataset converted and saved to: {yolo_folder}")

Converting Supervisely project to YOLO format: 100%|██████████| 211/211 [00:01<00:00, 118.16it/s]
{"message": "Dataset 'ds' has been converted to YOLO format.", "timestamp": "2025-06-30T01:48:09.328Z", "level": "info"}
{"message": "Project 'car-segmentation' has been converted to YOLO format.", "timestamp": "2025-06-30T01:48:09.329Z", "level": "info"}


Dataset converted and saved to: /root/yolo_dataset1


## Train

Train YOLO11 on [Detect](https://docs.ultralytics.com/tasks/detect/), [Segment](https://docs.ultralytics.com/tasks/segment/), [Classify](https://docs.ultralytics.com/tasks/classify/) and [Pose](https://docs.ultralytics.com/tasks/pose/) datasets. See [YOLO11 Train Docs](https://docs.ultralytics.com/modes/train/) for more information.

In [22]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n-seg.pt")  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data=yolo_yaml, epochs=10, imgsz=640, batch=32, workers=96, val=False)

Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/root/yolo_dataset1/data_config.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train7, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.

train: Scanning /root/yolo_dataset1/labels/train... 211 images, 0 backgrounds, 0 corrupt: 100%|██████████| 211/211 [00:00<00:00, 304.02it/s]

train: /root/yolo_dataset1/images/train/ds_003.png: 9 duplicate labels removed
train: /root/yolo_dataset1/images/train/ds_105618339-super-purple-modern-suv-car-side-view.png: 10 duplicate labels removed
train: /root/yolo_dataset1/images/train/ds_105618639-modern-pale-blue-suv-car-side-view.png: 9 duplicate labels removed
train: /root/yolo_dataset1/images/train/ds_156551_preview.png: 8 duplicate labels removed
train: /root/yolo_dataset1/images/train/ds_178583_Side_Profile_Web.png: 9 duplicate labels removed
train: /root/yolo_dataset1/images/train/ds_1965-Shelby-Cobra-Roadster-Side-Profile.png: 6 duplicate labels removed
train: /root/yolo_dataset1/images/train/ds_2008-Morgan-Life-Car-Side-Speed-1024x768.png: 6 duplicate labels removed
train: /root/yolo_dataset1/images/train/ds_2009-Ford-Kuga-Show-Car-Side-1280x960.png: 11 duplicate labels removed
train: /root/yolo_dataset1/images/train/ds_2011-278656-pagani-huayra-side-view1.png: 7 duplicate labels removed
train: /root/yolo_dataset1/imag

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 699.7±187.3 MB/s, size: 1017.6 KB)


val: Scanning /root/yolo_dataset1/labels/train.cache... 211 images, 0 backgrounds, 0 corrupt: 100%|██████████| 211/211 [00:00<?, ?it/s]

...

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):  50%|█████     | 2/4 [00:03<00:03,  1.83s/it]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:08<00:00,  2.13s/it]


                   all        211       1837          1      0.314      0.691      0.591          1      0.314      0.689      0.581
                   car        211        224          1      0.927      0.948       0.85          1      0.927      0.947      0.805
                 wheel        211        445          1       0.33      0.965       0.85          1       0.33      0.961       0.88
                lights        210        438          1          0    0.00023   0.000109          1          0   0.000683   0.000255
                window        211        730          1          0      0.851      0.664          1          0      0.849       0.64
Speed: 0.2ms preprocess, 2.8ms inference, 0.0ms loss, 7.1ms postprocess per image
Results saved to runs/segment/train7


![Dataset sample image](https://github.com/ultralytics/docs/releases/download/0/dataset-sample-image.avif)

## Predict

YOLO11 may be used directly in the Command Line Interface (CLI) with a yolo command for a variety of tasks and modes and accepts additional arguments, i.e. imgsz=640. See a full list of available [yolo arguments](https://docs.ultralytics.com/usage/cfg/) and other details in the [YOLO11 Predict Docs](https://docs.ultralytics.com/modes/train/).

In [23]:
from ultralytics import YOLO

# Load a model
model = YOLO(f"{model.trainer.save_dir}/weights/best.pt")  # load a fine-tuned model

# Inference using the model (img/video/stream)
prediction_results = model.predict("https://github.com/ultralytics/assets/releases/download/v0.0.0/carparts-image.jpg", save=True)

print(prediction_results)

100%|██████████| 336k/336k [00:00<00:00, 8.67MB/s]


image 1/1 /content/carparts-image.jpg: 384x640 (no detections), 182.7ms
Speed: 20.3ms preprocess, 182.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/segment/predict
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'car', 1: 'wheel', 2: 'lights', 3: 'window'}
obb: None
orig_img: array([[[38, 54, 71],
        [42, 62, 79],
        [51, 75, 93],
        ...,
        [12, 41, 46],
        [ 5, 35, 40],
        [13, 43, 48]],

       [[42, 58, 75],
        [47, 65, 82],
        [51, 76, 92],
        ...,
        [16, 45, 50],
        [ 9, 38, 43],
        [15, 45, 50]],

       [[40, 55, 71],
        [41, 60, 75],
        [43, 66, 81],
        ...,
        [21, 50, 54],
        [15, 44, 48],
        [21, 50, 54]],

       ...,

       [[26, 24, 24],
        [26, 24, 24],
        [26, 24, 24],
        ...,
        [24, 22, 22],
        [24, 22, 22],
  

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img align="left" src="https://github.com/user-attachments/assets/436ded05-9203-4bb7-883b-f7a1f9a399c1" width="600">

## Export

Export a YOLO11 model to any supported format below with the `format` argument, i.e. `format=onnx`. See [YOLO11 Export Docs](https://docs.ultralytics.com/modes/export/) for more information.

- 💡 ProTip: Export to [ONNX](https://docs.ultralytics.com/integrations/onnx/) or [OpenVINO](https://docs.ultralytics.com/integrations/openvino/) for up to 3x CPU speedup.  
- 💡 ProTip: Export to [TensorRT](https://docs.ultralytics.com/integrations/tensorrt/) for up to 5x GPU speedup.

| Format                                                                   | `format` Argument | Model                     | Metadata | Arguments                                                            |
|--------------------------------------------------------------------------|-------------------|---------------------------|----------|----------------------------------------------------------------------|
| [PyTorch](https://pytorch.org/)                                          | -                 | `yolo11n.pt`              | ✅        | -                                                                    |
| [TorchScript](https://docs.ultralytics.com/integrations/torchscript)     | `torchscript`     | `yolo11n.torchscript`     | ✅        | `imgsz`, `optimize`, `batch`                                         |
| [ONNX](https://docs.ultralytics.com/integrations/onnx)                   | `onnx`            | `yolo11n.onnx`            | ✅        | `imgsz`, `half`, `dynamic`, `simplify`, `opset`, `batch`             |
| [OpenVINO](https://docs.ultralytics.com/integrations/openvino)           | `openvino`        | `yolo11n_openvino_model/` | ✅        | `imgsz`, `half`, `dynamic`, `int8`, `batch`                          |
| [TensorRT](https://docs.ultralytics.com/integrations/tensorrt)           | `engine`          | `yolo11n.engine`          | ✅        | `imgsz`, `half`, `dynamic`, `simplify`, `workspace`, `int8`, `batch` |
| [CoreML](https://docs.ultralytics.com/integrations/coreml)               | `coreml`          | `yolo11n.mlpackage`       | ✅        | `imgsz`, `half`, `int8`, `nms`, `batch`                              |
| [TF SavedModel](https://docs.ultralytics.com/integrations/tf-savedmodel) | `saved_model`     | `yolo11n_saved_model/`    | ✅        | `imgsz`, `keras`, `int8`, `batch`                                    |
| [TF GraphDef](https://docs.ultralytics.com/integrations/tf-graphdef)     | `pb`              | `yolo11n.pb`              | ❌        | `imgsz`, `batch`                                                     |
| [TF Lite](https://docs.ultralytics.com/integrations/tflite)              | `tflite`          | `yolo11n.tflite`          | ✅        | `imgsz`, `half`, `int8`, `batch`                                     |
| [TF Edge TPU](https://docs.ultralytics.com/integrations/edge-tpu)        | `edgetpu`         | `yolo11n_edgetpu.tflite`  | ✅        | `imgsz`                                                              |
| [TF.js](https://docs.ultralytics.com/integrations/tfjs)                  | `tfjs`            | `yolo11n_web_model/`      | ✅        | `imgsz`, `half`, `int8`, `batch`                                     |
| [PaddlePaddle](https://docs.ultralytics.com/integrations/paddlepaddle)   | `paddle`          | `yolo11n_paddle_model/`   | ✅        | `imgsz`, `batch`                                                     |
| [MNN](https://docs.ultralytics.com/integrations/mnn)                     | `mnn`             | `yolo11n.mnn`             | ✅        | `imgsz`, `batch`, `int8`, `half`                                     |
| [NCNN](https://docs.ultralytics.com/integrations/ncnn)                   | `ncnn`            | `yolo11n_ncnn_model/`     | ✅        | `imgsz`, `half`, `batch`                                             |
| [IMX500](https://docs.ultralytics.com/integrations/sony-imx500)          | `imx`             | `yolov8n_imx_model/`      | ✅        | `imgsz`, `int8`                                                      |
| [RKNN](https://docs.ultralytics.com/integrations/rockchip-rknn)          | `rknn`            | `yolo11n_rknn_model/`     | ✅        | `imgsz`, `batch`, `name`                                             |

In [25]:
from ultralytics import YOLO

# Load a model

path= "runs/segment/train7"
model = YOLO(f"{path}/weights/best.pt")  # load a fine-tuned model

# Export the model
model.export(format="onnx")

Ultralytics 8.3.160 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.00GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel CPUs. Learn more at https://docs.ultralytics.com/integrations/openvino/
YOLO11n-seg summary (fused): 113 layers, 2,835,348 parameters, 0 gradients, 10.2 GFLOPs

PyTorch: starting from 'runs/segment/train7/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 40, 8400), (1, 32, 160, 160)) (5.7 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<1.18.0', 'onnxslim>=0.1.56', 'onnxruntime-gpu'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 9.8s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.58...
ONNX: export success ✅ 14.7s, saved as 'runs/segment/train7/weights/best.onnx' (11.1 MB)

Export complete (15.4s)
Results saved to /content/runs/segment/train7/weights
Pred

'runs/segment/train7/weights/best.onnx'